# Bootstrap Static Evolution Analysis

This notebook loads and analyzes results from bootstrap static evolution runs where one parameter is fixed and the other evolves.

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

# Set up plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Load Bootstrap Results

Find and load the most recent bootstrap results.

In [ ]:
# Find all bootstrap directories
bootstrap_dirs = glob.glob('bootstrap_static_evolution_*')
if not bootstrap_dirs:
    print("No bootstrap directories found!")
    print("Available directories:")
    print(os.listdir('.'))
else:
    # Use the most recent one
    bootstrap_dir = sorted(bootstrap_dirs)[-1]
    print(f"Using bootstrap directory: {bootstrap_dir}")
    
    # Load results
    results_file = f"{bootstrap_dir}/bootstrap_results.pkl"
    if os.path.exists(results_file):
        with open(results_file, 'rb') as f:
            data = pickle.load(f)
        
        results_g = data['results_g']
        results_s = data['results_s']
        config = data['config']
        
        print(f"Loaded results with {len(results_g)} fixed-g runs and {len(results_s)} fixed-s runs")
        print(f"Configuration: {config}")
    else:
        print(f"Results file not found: {results_file}")
        print(f"Available files in {bootstrap_dir}:")
        print(os.listdir(bootstrap_dir))

## Extract Data for Plotting

In [ ]:
def extract_plot_data(results, parameter_name):
    """Extract plotting data from results"""
    fixed_values = []
    evolved_means = []
    evolved_stds = []
    evolved_all_runs = []
    
    for result_group in results:
        fixed_val = result_group['fixed_value']
        runs = result_group['runs']
        
        if runs:  # If we have successful runs
            # Collect all run means for this fixed value
            run_means = [run['final_evolved_mean'] for run in runs]
            run_stds = [run['final_evolved_std'] for run in runs]
            
            fixed_values.append(fixed_val)
            evolved_means.append(np.mean(run_means))
            evolved_stds.append(np.std(run_means))  # Standard error across runs
            evolved_all_runs.append(run_means)
    
    return np.array(fixed_values), np.array(evolved_means), np.array(evolved_stds), evolved_all_runs

# Extract data for both directions
if 'results_g' in locals() and 'results_s' in locals():
    # Fixed g, evolved s
    fixed_g_vals, evolved_s_means, evolved_s_stds, s_all_runs = extract_plot_data(results_g, 'g')
    
    # Fixed s, evolved g  
    fixed_s_vals, evolved_g_means, evolved_g_stds, g_all_runs = extract_plot_data(results_s, 's')
    
    print(f"Fixed g data: {len(fixed_g_vals)} points")
    print(f"Fixed s data: {len(fixed_s_vals)} points")
else:
    print("No results loaded yet!")

## Main Bootstrap Plot

In [ ]:
if 'fixed_g_vals' in locals() and 'fixed_s_vals' in locals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Fixed g, evolved s
    if len(fixed_g_vals) > 0:
        ax1.errorbar(fixed_g_vals, evolved_s_means, yerr=evolved_s_stds, 
                    marker='o', capsize=5, capthick=2, linewidth=2, markersize=6)
        ax1.set_xlabel('Fixed w_g (Gradient Detection)', fontsize=12)
        ax1.set_ylabel('Final Evolved w_s (Sociality)', fontsize=12)
        ax1.set_title('Fixed Gradient Detection → Evolved Sociality', fontsize=14)
        ax1.grid(True, alpha=0.3)
        
        # Add individual run points
        for i, (fixed_val, runs) in enumerate(zip(fixed_g_vals, s_all_runs)):
            x_jitter = fixed_val + np.random.normal(0, 0.002, len(runs))
            ax1.scatter(x_jitter, runs, alpha=0.5, s=20, color='blue')
    
    # Plot 2: Fixed s, evolved g  
    if len(fixed_s_vals) > 0:
        ax2.errorbar(fixed_s_vals, evolved_g_means, yerr=evolved_g_stds,
                    marker='s', capsize=5, capthick=2, linewidth=2, markersize=6, color='orange')
        ax2.set_xlabel('Fixed w_s (Sociality)', fontsize=12)
        ax2.set_ylabel('Final Evolved w_g (Gradient Detection)', fontsize=12)
        ax2.set_title('Fixed Sociality → Evolved Gradient Detection', fontsize=14)
        ax2.grid(True, alpha=0.3)
        
        # Add individual run points
        for i, (fixed_val, runs) in enumerate(zip(fixed_s_vals, g_all_runs)):
            x_jitter = fixed_val + np.random.normal(0, 0.002, len(runs))
            ax2.scatter(x_jitter, runs, alpha=0.5, s=20, color='orange')
    
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    if 'bootstrap_dir' in locals():
        plot_file = f"{bootstrap_dir}/plots/bootstrap_analysis_notebook.png"
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {plot_file}")
else:
    print("No data available for plotting!")

## Detailed Analysis

In [ ]:
if 'fixed_g_vals' in locals() and 'fixed_s_vals' in locals():
    print("=== DETAILED ANALYSIS ===")
    
    print("\nFixed Gradient Detection (w_g) → Evolved Sociality (w_s):")
    for i, (fixed_val, mean_val, std_val) in enumerate(zip(fixed_g_vals, evolved_s_means, evolved_s_stds)):
        print(f"  w_g = {fixed_val:.3f} → w_s = {mean_val:.4f} ± {std_val:.4f}")
    
    print("\nFixed Sociality (w_s) → Evolved Gradient Detection (w_g):")
    for i, (fixed_val, mean_val, std_val) in enumerate(zip(fixed_s_vals, evolved_g_means, evolved_g_stds)):
        print(f"  w_s = {fixed_val:.3f} → w_g = {mean_val:.4f} ± {std_val:.4f}")
    
    # Look for trends
    print("\n=== TREND ANALYSIS ===")
    
    if len(fixed_g_vals) > 1:
        g_correlation = np.corrcoef(fixed_g_vals, evolved_s_means)[0,1]
        print(f"Correlation between fixed w_g and evolved w_s: {g_correlation:.4f}")
        
        # Simple linear fit
        g_slope = np.polyfit(fixed_g_vals, evolved_s_means, 1)[0]
        print(f"Linear slope (dw_s/dw_g): {g_slope:.4f}")
    
    if len(fixed_s_vals) > 1:
        s_correlation = np.corrcoef(fixed_s_vals, evolved_g_means)[0,1]
        print(f"Correlation between fixed w_s and evolved w_g: {s_correlation:.4f}")
        
        # Simple linear fit
        s_slope = np.polyfit(fixed_s_vals, evolved_g_means, 1)[0]
        print(f"Linear slope (dw_g/dw_s): {s_slope:.4f}")
else:
    print("No data available for analysis!")

## Evolution Trajectories (Optional)

Load and plot evolution trajectories for specific parameter values.

In [ ]:
def load_evolution_trajectory(checkpoint_dir):
    """Load evolution history from a checkpoint directory"""
    history_file = f"{checkpoint_dir}/complete_history.pkl"
    if os.path.exists(history_file):
        with open(history_file, 'rb') as f:
            data = pickle.load(f)
        return data['history']
    return None

# Plot a few example trajectories
if 'results_g' in locals() and len(results_g) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot trajectories for fixed g (first few values)
    for i, result_group in enumerate(results_g[:3]):  # First 3 fixed values
        fixed_val = result_group['fixed_value']
        if result_group['runs']:
            run = result_group['runs'][0]  # First run
            checkpoint_dir = run['checkpoint_dir']
            
            history = load_evolution_trajectory(checkpoint_dir)
            if history:
                generations = history['generation']
                mean_s = history['mean_w_s']
                ax1.plot(generations, mean_s, label=f'Fixed w_g = {fixed_val:.3f}', linewidth=2)
    
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Mean w_s (Sociality)')
    ax1.set_title('Evolution of Sociality with Fixed Gradient Detection')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot trajectories for fixed s (first few values)
    for i, result_group in enumerate(results_s[:3]):  # First 3 fixed values
        fixed_val = result_group['fixed_value']
        if result_group['runs']:
            run = result_group['runs'][0]  # First run
            checkpoint_dir = run['checkpoint_dir']
            
            history = load_evolution_trajectory(checkpoint_dir)
            if history:
                generations = history['generation']
                mean_g = history['mean_w_g']
                ax2.plot(generations, mean_g, label=f'Fixed w_s = {fixed_val:.3f}', linewidth=2)
    
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Mean w_g (Gradient Detection)')
    ax2.set_title('Evolution of Gradient Detection with Fixed Sociality')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Summary

This analysis shows how fixing one behavioral parameter affects the evolution of the other parameter. Key insights:

1. **Trade-offs**: Look for negative correlations between fixed and evolved parameters
2. **Complementarity**: Look for positive correlations indicating synergistic effects
3. **Threshold effects**: Look for non-linear relationships or plateaus
4. **Evolutionary constraints**: Compare the range of evolved values to understand selection pressures